In [0]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

W0614 20:58:44.313683 140002422785920 deprecation.py:323] From <ipython-input-2-e99a00302c11>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0614 20:58:44.315421 140002422785920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0614 20:58:44.316620 140002422785920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0614 20:58:45.361142 140002422785920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0614 20:58:45.365797 140002422785920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz


W0614 20:58:45.764813 140002422785920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width // (max_pool1_k * max_pool2_k)) * (input_height // (max_pool1_k * max_pool2_k)) * n_conv2

In [0]:
weights = {
    "wc1": tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2": tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1": tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    "wo":tf.Variable(tf.random_normal([n_hidden,n_out]))
}

biases = {
    "bc1": tf.Variable(tf.random_normal([n_conv1])),
    "bc2": tf.Variable(tf.random_normal([n_conv2])),
    "bh1": tf.Variable(tf.random_normal([n_hidden])),
    "bo":tf.Variable(tf.random_normal([n_out]))    
}

In [0]:
def conv(x,weights,bias,strides = 1):
    out = tf.nn.conv2d(x,weights,padding = "SAME",strides = [1,strides,strides,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out

In [0]:
def maxpooling(x,k = 2):
    return tf.nn.max_pool(x,padding = "SAME",ksize = [1,k,k,1],strides = [1,k,k,1])

In [0]:
def cnn(x,weights,biases):
    x = tf.reshape(x,shape = [-1,input_height,input_width,input_channels])
    conv1 = conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape = [-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1']) 
    hidden_output = tf.nn.relu(hidden_output_before_activation) 
    
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output    

In [0]:
x = tf.placeholder("float",[None, input_pixels])
y = tf.placeholder(tf.int32,[None,n_out])
pred = cnn(x,weights,biases)

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred,labels = y))

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        c,_ = sess.run([cost,optimize],feed_dict = {x:batch_x,y:batch_y})
        total_cost += c
    print(total_cost)

1357724.824439615
26608.651469101904
14631.028211262077
14411.893501912324
9825.592259904237
10784.600190835712
9907.461700665353
11281.78856536357
9343.354655656538
8292.506968870759
6548.645118279905
6732.415277719556
6124.1375513920175
5888.853241329649
4891.21293487689
4950.241173632712
3655.336162754506
4553.717466537899
3419.523401746594
2799.70285143553
3603.678968522789
3648.244343190999
2432.840978440525
2714.134570063104
2464.898001664071


In [14]:
predictions = tf.argmax(pred,1)
correct_labels = tf.argmax(y,1)
correct_predictions = tf.equal(predictions,correct_labels)
predictions,correct_predictions = sess.run([predictions,correct_predictions],feed_dict = {x:mnist.test.images,y:mnist.test.labels})
correct_predictions.sum()

9868